## Python Imports

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from agents import Agent, Runner, trace, function_tool
from typing import Dict
import asyncio
import gradio as gr
import json
import httpx
# ruff: noqa: F704

## Load OpenAI Key

This key is used to call OpenAI models. You can use any other model as well.

At the end it is just a API call with appropriate credentianls. 

This SDK and frameworks are not tied or coupled to one model.

In [2]:
load_dotenv(override=True)

True

In [3]:
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
USE_PROXY = True  # Set to True to use a SOCKS5 proxy, False otherwise

In [5]:
if USE_PROXY:
    SOCKS_PROXY = "socks5h://127.0.0.1:9999"

    try:
        with httpx.Client(proxy=SOCKS_PROXY, timeout=10) as c:
            r = c.get("https://api.openai.com/v1/models")
            print("STATUS:", r.status_code)
            print(r.text[:200])
    except Exception as e:
        print("ERROR:", type(e).__name__, e)


STATUS: 401
{
  "error": {
    "message": "Missing bearer authentication in header",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}


In [6]:
MODEL = "gpt-4o-mini"

In [7]:
if USE_PROXY:
    http_client = httpx.Client(
        proxy=SOCKS_PROXY,
        timeout=30.0,
    )

In [8]:
if USE_PROXY:
    client = OpenAI(
        http_client=http_client
        )
else:
    client = OpenAI()

## Example of using any other model or proxy

In [9]:
# open_router_api_key = os.getenv('OPENROUTER_API_KEY')

# if not open_router_api_key:
#     print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not open_router_api_key.startswith("sk-or-v1-"):
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
# elif open_router_api_key.strip() != open_router_api_key:
#     print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
# else:
#     print("API key found and looks good so far!")

In [10]:
# client= OpenAI(
#     base_url="https://openrouter.ai/api/v1",
#     api_key=open_router_api_key
# )
# MODEL = "gpt-4o-mini"

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

## Types of prompts


Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**System prompt** -- tells them what task they are performing and what tone they should use

**User prompt** -- the conversation starter that they should reply to 


## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)


In [11]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = client.chat.completions.create(model=MODEL, messages=messages)
print(response.choices[0].message.content)

Oh, let me dust off my calculator for this one... It’s 4. Shocking, I know!


In [ ]:
messages = [
    {"role": "system", "content": "You are a polite assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
response = client.chat.completions.create(model=MODEL, messages=messages)
print(response.choices[0].message.content)

## Flight AI Assistant

Let us work with an example of Flight AI Assistant of a fictitious airline company.

We will take this example to demostrate the concepts we learn.

We will create some simple use cases just to keep the same example going. Helps to understand things better


### Memory

We already saw how memory is important for maintaining conversations and giving LLM some sort of context on what is going on

The SDKs have many ways to implement this, including persistent memory based on user sessions, and authentication.

Here we look into the very simple way of implementing memory. SDKs have more sophisticated ways of doing this.

At the end, you need to provide the memory in form of context to the LLM

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += ("Always be accurate. If you don't know the answer, say so. Do not make up an answer "
                  "or attempt to use previous knowledge apart from what is provided to you")

**Without Adding the History**

In [ ]:
def chat(message, history):
    print("\n💬 New message:", message)
    print("🧠 History:", history)
    messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    print("\n📨 Final Message:", messages)
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages",js=force_dark_mode).launch()

**Adding the History**

In [ ]:
def chat(message, history):
    print("\n💬 New message:", message)
    print("🧠 History:", history)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print("\n📨 Final Message:", messages)
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages",js=force_dark_mode).launch()

## TOOLS

We saw that tools are a way to provide the LLM with capabilities

But Remember, tools have to be executed by host. LLMs don't do that

They can tell you when to execute and how to execute only!

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += ("Always be accurate. If you don't know the answer, say so. Do not make up an answer "
                  "or attempt to use previous knowledge apart from what is provided to you")

In [ ]:
print(system_message)

In [ ]:
def chat(message, history):
    print("\n💬 New message:", message)
    print("🧠 History:", history)
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print("\n📨 Final Message:", messages)
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages",js=force_dark_mode).launch()

We are creating a dummy function that returns the price of a ticket to the 4 cities below

- London: USD 799
- Paris: USD 899
- Tokyo : USD 1400
- Berlin: USD 499

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Berlin")

Now we will define a python object that will represent this tool.

Look carefully at the structure of the object

It has a name, description, parameters required along with their types and description

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

You have to create this tools array that containes the tool objects. You can have multiple tools

See that the `price_function` is the definition of the function that we defined and LLM will tell you if this tool needs to be executed

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        print("\nOpenAI Decides: \n", response.choices[0].finish_reason)
        message = response.choices[0].message
        print("\nOpenAI Describes: \n", response.choices[0].message)
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print("Messages after tool call: ", messages)
        response = client.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    
    print("\nTool Call is being handled by the host:\n", tool_call)
    
    arguments = json.loads(tool_call.function.arguments)
    
    city = arguments.get('destination_city')

    ## Execute the function
    price = get_ticket_price(city)
    
    print("\nCreating a response for sending to the LLM\n")
    
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages",js=force_dark_mode).launch()

## Agents

We say what is an agent and how it comprises of LLM, tools, memory.

We also saw how an Agent loop is designed. Every SDK provides multiple ways to implement this.

Every SDK has a way to define tools. Here is an example of how you might define a tool in OpenAI SDK

As you can see the SDK provides a decortator to define a tool. This takes care of creating all the boilerplate code for a tool definition we saw earlier

***Notice the function docstring and type Annotations***

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
@function_tool
def get_ticket_price(destination_city: str) -> str:
    """Get the price of a return ticket to the destination city. 
    Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'
    """
    print(f"Tool get_ticket_price called for {destination_city} by the Agent")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
print(get_ticket_price)

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += ("Always be accurate. If you don't know the answer, say so. Do not make up an answer "
                  "or attempt to use previous knowledge apart from what is provided to you")

In [ ]:
agent = Agent(
    name="FlightAI",
    instructions=(
        system_message
    ),
    tools=[get_ticket_price],
    model="gpt-4o-mini",
)

with trace("FlightAIAgent"):
    result = await (Runner.run(agent, input="How much is the ticket price to Berlin"))
    print(result.final_output)

In [ ]:
async def chat_async(message, history):
    with trace("FlightAIChat"):
        print("\n📨 Agent Called with Message:", message)
        result = await Runner.run(agent, message)
        return result.final_output

In [ ]:
gr.ChatInterface(fn=chat_async, type="messages",js=force_dark_mode).launch()

## Memory Management with SDKs and Frameworks

Memory is provided to the agent simply by taking the history of messages and providing it as context to the Agent.


However in reality the SDKs or Frameworks you use provide many ways to implement memory, also persistent memory based on user sessions and authentication

In [ ]:
async def chat_async(message, history):
    print("\n💬 New message:", message)
    print("🧠 History:", history)
    items = [{"role":m.get("role"),"content":m.get("content")} for m in history if isinstance(m.get("content"), str)]
    items.append({"role": "user", "content": message})
    print("\n📨 Final Message:", items)
    with trace("FlightAIChatMemory"):
        result = await Runner.run(agent, items)
        return result.final_output

In [ ]:
gr.ChatInterface(fn=chat_async, type="messages",js=force_dark_mode).launch()

## MCP

As we have seen MCP is way to provide Agents or LLMS with more capabilities using tools hosted by somewhere else than the Agent Environment

We will first see client and server mechanism of MCP that does not involve any LLM or Agent.

We will understand that it is standard similar to REST and clearly defines how a MCP client and server interact

This standardizes the way All Agents are able to interact with any MCP compliant server and utilize their tools

SDKs implement the MCP standard in their own way. But the underlying mechanism is the same across all implementations.

`/list_tools` -- GET -- returns the list of tools the server has

`/call_tool` -- POST -- call a tool with parameters

Here we are using again OpenAI SDK but you can use any other SDK or framework that supports MCP

In [ ]:
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

Tell SDK how to connect to mcp server. Here we are using stdio transport but you can also use HTTP with SSE.

In the OpenAI SDK you can use `MCPServerStdio` or `MCPServerHTTP` to connect to the server. This internally creates the MCP client and connects to the server

In [ ]:
# Tell server to launch as a subprocess

params = {"command": "uv", "args": ["run", "mcp_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [ ]:
mcp_tools

In [ ]:
params = {"command": "uv", "args": ["run", "mcp_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    price = await server.call_tool(tool_name="get_ticket_price", arguments={"destination_city": "Berlin"})

In [ ]:
price

As you saw there is no LLM, it is just a way for a client and server to interact with set protocols and standards

## Creating your own MCP Client


Above we saw how MCP works without LLM or Agent using SDK. It creates a client and connects to the server.

Lets see how you can create your own MCP Client.

Here we are not using OpenAI SDK but creating our own MCP Client using MCP Python package from Anthropic

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from contextlib import AsyncExitStack
from mcp.types import TextContent

In [ ]:
server_params = StdioServerParameters(command="uv", args=["run", "mcp_server.py"])
async with AsyncExitStack() as stack:
    try:
        # Establish a stdio connection to the server using the server parameters
        transport = await stack.enter_async_context(stdio_client(server_params))
        read, write, *_ = transport
        
        # Create a client session using the read and write streams from the connection
        session = await stack.enter_async_context(ClientSession(read, write))
        # Initialize the session (e.g., perform handshake or setup operations)
        await session.initialize()
        print("\n✅ Connected to server. Checking Tools...")

        # Load the MCP tools from the connected server using the adapter function
        response = await session.list_tools()

        # Iterate over each tool and add it to the aggregated tools list
        tools = response.tools
        print("\n✅ Tools Discovered from MCP Server:~\n")
        for tool in tools:
            print(tool)
        print("\n✅ Tool:", [tool.name for tool in tools])

    except Exception as e:
      # Handle any errors that occur during connection or tool loading for the server
      print(f"❌ Failed to connect to server : {e}")

    # call_tool: get_ticket_price
    print("\nCalling tool get_ticket_price from the MCP server for 'Berlin'")

    price_res = await session.call_tool(
        name="get_ticket_price",
        arguments={"destination_city": "Berlin"},
    )
    price_text = next((c.text for c in (price_res.content or []) if isinstance(c, TextContent)), None)
    print("\n✅ Received Response from tool get_ticket_price('Berlin') ->", price_text or price_res)

    #call_tool: convert_usd_to_eur (optional, if your server implements it)
    print(f"\nCalling tool convert_usd_to_eur from the MCP server for {price_text}")
    eur_res = await session.call_tool(
                name="convert_usd_to_eur",
                arguments={"price_usd": price_text, "rate": 0.92},
                )
    eur_text = next((c.text for c in (eur_res.content or []) if isinstance(c, TextContent)), None)
    print("\n✅ Received Response from tool convert_usd_to_eur('$499', 0.92) ->", eur_text or eur_res)
    

### 

### ---

In [ ]:
# from mcp import ClientSession, StdioServerParameters
# from mcp.client.stdio import stdio_client
# from contextlib import AsyncExitStack
# from mcp.types import TextContent
# from agents.mcp import MCPServerStdio

In [ ]:
# server_params = [{"command": "uv", "args":["run", "mcp_server.py"]}]
# async with AsyncExitStack() as stack:
#     # build & connect servers (RETURN actual server objects, not coroutines)
#     mcp_servers = [
#         await stack.enter_async_context(
#             MCPServerStdio(
#                 params=p,
#                 name=f"stdio-{i}",
#                 client_session_timeout_seconds=120,
#                 # optional:
#                 # cache_tools_list=True,
#                 # use_structured_content=True,
#             )
#         )
#         for i, p in enumerate(server_params)
#     ]

#     agent = Agent(
#         name="FlightAI",
#         instructions=(
#             system_message
#         ),
#         #tools=[get_ticket_price],
#         mcp_servers=mcp_servers,
#         model="gpt-4o-mini",
#         )
#     with trace("FlightAIMCP"):
#         result = await (Runner.run(agent, input="How much is the ticket price to Berlin. And convert it to Euros too, please."))
#         print(result.final_output)

In [ ]:
# server_params = [{"command": "uv", "args":["run", "mcp_server.py"]}]
# #async with AsyncExitStack() as stack:
# stack = AsyncExitStack()
# await stack.__aenter__() 
#     # build & connect servers (RETURN actual server objects, not coroutines)
# mcp_servers = [
#     await stack.enter_async_context(
#         MCPServerStdio(
#             params=server_param,
#             name=f"stdio-{i}",
#             client_session_timeout_seconds=120,
#             # optional:
#             # cache_tools_list=True,
#             # use_structured_content=True,
#         )
#     )
#     for i, server_param in enumerate(server_params)
# ]

# agent = Agent(
#     name="FlightAI",
#     instructions=(
#         system_message
#     ),
#     #tools=[get_ticket_price],
#     mcp_servers=mcp_servers,
#     model="gpt-4o-mini",
#     )
#     #result = await (Runner.run(agent, input="How much is the ticket price to Berlin. And convert it to Euros too, please."))
#     #print(result.final_output)
    
# async def chat_mcp(message, history):
#         print(history)
#         print(message)
#         items = [{"role":m.get("role"),"content":m.get("content")} for m in history if isinstance(m.get("content"), str)]
#         items.append({"role": "user", "content": message})
#         with trace("FlightAIMCPChat"):
#             result = await Runner.run(agent, items)
#             return result.final_output

# gr.ChatInterface(fn=chat_mcp, type="messages",js=force_dark_mode).launch()

In [ ]:
# await stack.aclose()

In [ ]:
# server_params = [{"command": "uv", "args":["run", "mcp_server.py"]}]
# #async with AsyncExitStack() as stack:
# async with AsyncExitStack() as stack:
#     mcp_servers = [
#         await stack.enter_async_context(
#             MCPServerStdio(
#                 params=server_param,
#                 name=f"stdio-{i}",
#                 client_session_timeout_seconds=120,
#                 # optional:
#                 # cache_tools_list=True,
#                 # use_structured_content=True,
#             )
#         )
#         for i, server_param in enumerate(server_params)
#     ]
    
#     agent = Agent(
#         name="FlightAI",
#         instructions=(
#             system_message
#         ),
#         #tools=[get_ticket_price],
#         mcp_servers=mcp_servers,
#         model="gpt-4o-mini",
#         )
#         #result = await (Runner.run(agent, input="How much is the ticket price to Berlin. And convert it to Euros too, please."))
#         #print(result.final_output)
        
#     async def chat_mcp(message, history):
#             print(history)
#             print(message)
#             items = [{"role":m.get("role"),"content":m.get("content")} for m in history if isinstance(m.get("content"), str)]
#             items.append({"role": "user", "content": message})
#             result = await Runner.run(agent, items)
#             return result.final_output
    
#     gr.ChatInterface(fn=chat_mcp, type="messages",js=force_dark_mode).launch()

#     try:
#         while True:
#             await asyncio.sleep(3600)
#     # except (KeyboardInterrupt, SystemExit):
#     except:
#         pass